# **IND320 Project Log** - Assignement 3 : Electricity Production Data Analysis
**Esteban Carrasco**  
*November 07, 2025*

---

## **1. Project Overview**

This project aimed to analyze **hourly electricity production data** from Elhub for Norwegian price areas using Python (Pandas, NumPy, SciPy, Plotly) and deploy interactive visualizations. The dataset, provided in CSV format, required preprocessing for temporal analysis and multi-scale visualization due to varying production types (hydro, wind, etc.).

#### Links

* **Streamlit App** : [see here](https://ind320-projectwork-esteban-carrasco.streamlit.app)
* **Github** : [see here](https://github.com/Ficus22/IND320-ProjectWork)

---

## **2. Development Process**

### **2.1 Log:**

**Objective**:
Analyze hourly electricity production in Norway (price areas NO1–NO5) for 2019–2021, focusing on production groups such as hydro and wind. The goal was to create reproducible visualizations, detect anomalies, and generate interactive plots including seasonal decomposition and spectrograms.

---

#### **A- Data Preparation**

1. **Price Areas and Locations**:
   Created a Pandas DataFrame containing the five electricity price areas: **Oslo, Kristiansand, Trondheim, Tromsø, Bergen**, along with their longitude and latitude coordinates (geographical center points).

2. **Data Loading and Cleaning**:

   * Loaded CSV production data using Pandas.
   * Ensured `start_time` was converted to datetime for indexing.
   * Checked for missing or duplicate values.
   * Filtered by price area and production group for analysis.

**Challenge**:
Column names sometimes contained extra spaces; added stripping of whitespace to prevent key errors (`KeyError: 'price_area'`).

---

#### **B- Outlier Detection and Anomaly Analysis**

1. **Temperature Analysis for Bergen** (historical weather data via open-meteo API):

   * Downloaded ERA5 reanalysis data for 2019 using a custom function that takes longitude, latitude, and year as input.
   * Plotted raw hourly temperature.
   * Applied Direct Cosine Transform (DCT) high-pass filtering to create seasonally adjusted temperature variations (SATV).
   * Used robust statistics to define control limits and highlight outliers.

2. **Precipitation Analysis**:

   * Plotted hourly precipitation.
   * Detected anomalies using the Local Outlier Factor (LOF) method.
   * Parameterized the proportion of outliers (default 1%).

**Outcome**:
Both functions returned plots and summaries of detected anomalies. Outliers were visually highlighted in contrasting colors.

---

#### **C- Seasonal Decomposition (STL)**

* Performed STL decomposition on Elhub production data.
* Parameters included price area, production group, seasonal period, seasonal smoother, trend smoother, and robust option.
* Plotted observed, trend, seasonal, and residual components.
* Wrapped in a function returning the decomposition figure for testing and reproducibility.

---

#### **D- Spectrogram Analysis**

* Implemented a spectrogram function using SciPy and Plotly to visualize periodic patterns in electricity production.
* Parameters: price area, production group, window length, window overlap, and colorscale.
* Handled missing timestamps with interpolation.
* Returned an interactive Plotly heatmap of power spectral density over time.

**Example Output**:

* Bergen hydro production spectrogram revealed daily and weekly cycles, with stronger intensity during winter months.

---

#### **E- Function Wrapping and Testing**

* Each analysis step (outliers, STL decomposition, spectrogram) was encapsulated in reusable Python functions.
* Functions were tested with default parameters and plotted for selected areas (Bergen, NO1).

---

#### **F- Challenges and Solutions**

| Issue                                      | Solution                                                              |
| ------------------------------------------ | --------------------------------------------------------------------- |
| KeyError on column access                  | Stripped whitespace from CSV headers                                  |
| Missing timestamps in hourly data          | Interpolated missing hours with `time` method                         |
| Large spectrogram arrays slowing rendering | Optimized with smaller window overlaps and `zsmooth="best"` in Plotly |
| Seasonal trend affecting outlier detection | Used high-pass DCT to isolate short-term anomalies                    |

**Collaboration**:
Shared code snippets with classmates to validate STL parameters and frequency analysis approach.

---

### **2.2 AI Assistance**:

*Le Chat* ([Mistral AI](https://mistral.ai/)) helped optimize Pandas data filtering and datetime handling, suggested approaches for spectrogram computation, and provided guidance on anomaly detection using DCT and LOF. It also helped translate project documentation into English.

---

## **3. Jupyter Notebook Phase**


In [1]:
%matplotlib inline

In [2]:
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

## Elhub API

Testing API connection

In [4]:
import requests

entity ="price-areas"
dataset = "PRODUCTION_PER_GROUP_MBA_HOUR"
URL = f"https://api.elhub.no/energy-data/v0/{entity}?dataset={dataset}"
response = requests.get(URL)

print(response.status_code)

200


Fetching data from Elhub API

In [5]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import requests
import pandas as pd

ENTITY = "price-areas"
DATASET_PRODUCTION = "PRODUCTION_PER_GROUP_MBA_HOUR"
YEARS_PRODUCTION = [2022, 2023, 2024]

def generate_monthly_ranges(year):
    start_year = datetime(year, 1, 1)
    end_year = datetime(year, 12, 31)
    ranges = []
    current = start_year
    while current <= end_year:
        month_start = current
        month_end = (current + relativedelta(months=1)) - relativedelta(seconds=1)
        start_str = month_start.strftime("%Y-%m-%dT%H:%M:%S") + "%2B01:00"
        end_str = month_end.strftime("%Y-%m-%dT%H:%M:%S") + "%2B01:00"
        ranges.append((start_str, end_str))
        current += relativedelta(months=1)
    return ranges

all_production_records = []

for year in YEARS_PRODUCTION:
    monthly_ranges = generate_monthly_ranges(year)
    for start_date, end_date in monthly_ranges:
        url = f"https://api.elhub.no/energy-data/v0/{ENTITY}?dataset={DATASET_PRODUCTION}&startDate={start_date}&endDate={end_date}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            for entry in data.get("data", []):
                records = entry.get("attributes", {}).get("productionPerGroupMbaHour", [])
                all_production_records.extend(records)
            #print(f"Fetched {len(records)} records for {start_date[:10]} ({year}).")
        else:
            print(f"Error {response.status_code} for {start_date[:10]} ({year}).")

print(f"Total production records fetched (2022-2024): {len(all_production_records)}")


Total production records fetched (2022-2024): 657600


In [5]:
# display the head
for i, record in enumerate(all_production_records[:5]):
    print(f"Record {i+1}:")
    print(record)
    print("-" * 50)

Record 1:
{'endTime': '2022-01-01T01:00:00+01:00', 'lastUpdatedTime': '2025-02-01T18:02:57+01:00', 'priceArea': 'NO1', 'productionGroup': 'hydro', 'quantityKwh': 1291422.4, 'startTime': '2022-01-01T00:00:00+01:00'}
--------------------------------------------------
Record 2:
{'endTime': '2022-01-01T02:00:00+01:00', 'lastUpdatedTime': '2025-02-01T18:02:57+01:00', 'priceArea': 'NO1', 'productionGroup': 'hydro', 'quantityKwh': 1246209.4, 'startTime': '2022-01-01T01:00:00+01:00'}
--------------------------------------------------
Record 3:
{'endTime': '2022-01-01T03:00:00+01:00', 'lastUpdatedTime': '2025-02-01T18:02:57+01:00', 'priceArea': 'NO1', 'productionGroup': 'hydro', 'quantityKwh': 1271757.0, 'startTime': '2022-01-01T02:00:00+01:00'}
--------------------------------------------------
Record 4:
{'endTime': '2022-01-01T04:00:00+01:00', 'lastUpdatedTime': '2025-02-01T18:02:57+01:00', 'priceArea': 'NO1', 'productionGroup': 'hydro', 'quantityKwh': 1204251.8, 'startTime': '2022-01-01T03:0

## Adding data into MongoDB

Connection test

In [6]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Get the URI from environment variables
uri = os.getenv("MONGO_URI")

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


Transform API records to match existing collection format using a dataframe to be faster

In [16]:
import pandas as pd

# Créer un DataFrame directement depuis la liste de dicts
df = pd.DataFrame(all_production_records)

#print(df.head())

# Rename columns and keep only the necessary ones
df = df.rename(columns={
    "priceArea": "price_area",
    "productionGroup": "production_group",
    "startTime": "start_time",
    "quantityKwh": "quantity_kwh"
})[["price_area", "production_group", "start_time", "quantity_kwh"]]



print(df.dtypes)
print(df.head())

price_area           object
production_group     object
start_time           object
quantity_kwh        float64
dtype: object
  price_area production_group                 start_time  quantity_kwh
0        NO1            hydro  2022-01-01T00:00:00+01:00     1291422.4
1        NO1            hydro  2022-01-01T01:00:00+01:00     1246209.4
2        NO1            hydro  2022-01-01T02:00:00+01:00     1271757.0
3        NO1            hydro  2022-01-01T03:00:00+01:00     1204251.8
4        NO1            hydro  2022-01-01T04:00:00+01:00     1202086.9


In [ ]:
from pymongo import MongoClient
from pymongo.server_api import ServerApi
from dotenv import load_dotenv
import os
import pandas as pd

# Connect to MongoDB
load_dotenv()
client = MongoClient(os.getenv("MONGO_URI"), server_api=ServerApi('1'))
db = client["elhub_data"]
collection = db["production_data"]

# Convert the DataFrame to a dictionary and insert
data_for_mongo = df.to_dict("records")
collection.insert_many(data_for_mongo)
print(f"{len(data_for_mongo)} documents inserted into MongoDB.")


657600 documents inserted into MongoDB.


## Adding data into Cassandra table

Connect to Cassandra

In [31]:
from cassandra.cluster import Cluster
from cassandra.concurrent import execute_concurrent_with_args
import sys

KEYSPACE = "my_ind320_keyspace"
TABLE = "elhub_data"
CASSANDRA_HOST = "localhost"
CASSANDRA_PORT = 9042
CONCURRENCY_LEVEL = 100  # number of simultaneous inserts

try:
    cluster = Cluster([CASSANDRA_HOST], port=CASSANDRA_PORT)
    session = cluster.connect()
    print("✅ Connected to Cassandra.")
except Exception as e:
    sys.exit(f"❌ Cassandra connection error: {e}")

SystemExit: ❌ Cassandra connection error: ('Unable to connect to any servers', {'127.0.0.1:9042': OperationTimedOut('errors=Timed out creating connection (5 seconds), last_host=None'), '::1:9042': ConnectionShutdown('Connection to ::1:9042 was closed')})

Connection test

In [ ]:
session.set_keyspace('my_ind320_keyspace')
rows = session.execute("SELECT table_name FROM system_schema.tables WHERE keyspace_name = 'my_ind320_keyspace';")
for row in rows:
    print(row.table_name)

NoHostAvailable: ('Unable to complete the operation against any hosts', {})

Import data

In [27]:
from tqdm import tqdm

session.set_keyspace(KEYSPACE)

insert_query = session.prepare(f"""
    INSERT INTO {TABLE} (price_area, production_group, start_time, quantity_kwh)
    VALUES (?, ?, ?, ?)
""")


print("🚀 Inserting data into Cassandra...")

params = [
    (
        row["price_area"],
        row["production_group"],
        row["start_time"].to_pydatetime(),
        float(row["quantity_kwh"])
    )
    for _, row in df.iterrows()
]

# Use tqdm for progress tracking
results = list(
    tqdm(
        execute_concurrent_with_args(
            session, insert_query, params, concurrency=CONCURRENCY_LEVEL
        ),
        total=len(params),
        desc="Insertion"
    )
)

# Check for potential errors
errors = [res for res in results if not res[0]]
if errors:
    print(f"⚠️ {len(errors)} insertion errors detected.")
else:
    print("✅ All data inserted successfully!")


cluster.shutdown()
print("\n🏁 Import completed.")

NoHostAvailable: ('Unable to complete the operation against any hosts', {})

## Adding a new table into databases

Fetching data from Elhub API

In [6]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import requests
import pandas as pd

# CONSTANTS FOR CONSUMPTION
ENTITY = "price-areas"
DATASET_CONSUMPTION = "CONSUMPTION_PER_GROUP_MBA_HOUR"
YEARS_CONSUMPTION = [2021, 2022, 2023, 2024]


# FUNCTION TO GENERATE MONTHLY DATE RANGES
def generate_monthly_ranges(year):
    start_year = datetime(year, 1, 1)
    end_year = datetime(year, 12, 31)
    ranges = []
    current = start_year
    while current <= end_year:
        month_start = current
        month_end = (current + relativedelta(months=1)) - relativedelta(seconds=1)
        # Formatting for URL encoding +01:00
        start_str = month_start.strftime("%Y-%m-%dT%H:%M:%S") + "%2B01:00"
        end_str = month_end.strftime("%Y-%m-%dT%H:%M:%S") + "%2B01:00"
        ranges.append((start_str, end_str))
        current += relativedelta(months=1)
    return ranges


# FETCH HOURLY CONSUMPTION DATA 2021 - 2024
all_consumption_records = []

for year in YEARS_CONSUMPTION:
    monthly_ranges = generate_monthly_ranges(year)
    for start_date, end_date in monthly_ranges:
        url = f"https://api.elhub.no/energy-data/v0/{ENTITY}?dataset={DATASET_CONSUMPTION}&startDate={start_date}&endDate={end_date}"
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            for entry in data.get("data", []):
                records = entry.get("attributes", {}).get("consumptionPerGroupMbaHour", [])
                all_consumption_records.extend(records)
        else:
            print(f"Error {response.status_code} for {start_date[:10]} ({year}).")

print(f"Total consumption records fetched (2021–2024): {len(all_consumption_records)}")


Total consumption records fetched (2021–2024): 876600


In [7]:
# display the head
for i, record in enumerate(all_consumption_records[:5]):
    print(f"Record {i+1}:")
    print(record)
    print("-" * 50)

Record 1:
{'consumptionGroup': 'cabin', 'endTime': '2021-01-01T01:00:00+01:00', 'lastUpdatedTime': '2024-12-20T10:35:40+01:00', 'meteringPointCount': 100607, 'priceArea': 'NO1', 'quantityKwh': 177071.56, 'startTime': '2021-01-01T00:00:00+01:00'}
--------------------------------------------------
Record 2:
{'consumptionGroup': 'cabin', 'endTime': '2021-01-01T02:00:00+01:00', 'lastUpdatedTime': '2024-12-20T10:35:40+01:00', 'meteringPointCount': 100607, 'priceArea': 'NO1', 'quantityKwh': 171335.12, 'startTime': '2021-01-01T01:00:00+01:00'}
--------------------------------------------------
Record 3:
{'consumptionGroup': 'cabin', 'endTime': '2021-01-01T03:00:00+01:00', 'lastUpdatedTime': '2024-12-20T10:35:40+01:00', 'meteringPointCount': 100607, 'priceArea': 'NO1', 'quantityKwh': 164912.02, 'startTime': '2021-01-01T02:00:00+01:00'}
--------------------------------------------------
Record 4:
{'consumptionGroup': 'cabin', 'endTime': '2021-01-01T04:00:00+01:00', 'lastUpdatedTime': '2024-12-2

#### In MongoDB

In [ ]:
import pandas as pd

df_cons = pd.DataFrame(all_consumption_records)

# Keep only needed fields and rename to target format
df_cons = df_cons.rename(columns={
    "priceArea": "price_area",
    "consumptionGroup": "consumption_group",
    "startTime": "start_time",
    "quantityKwh": "quantity_kwh",
    "meteringPointCount": "metering_point_count"
})[[
    "price_area",
    "consumption_group",
    "start_time",
    "quantity_kwh",
    "metering_point_count"
]]

# Convert start_time to datetime
df_cons["start_time"] = pd.to_datetime(df_cons["start_time"], utc=True)

print(df_cons.dtypes)
print(df_cons.head())

C:\Users\Esteban\AppData\Local\Temp\ipykernel_10180\2678956536.py:21: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df_cons["start_time"] = pd.to_datetime(df_cons["start_time"])


price_area               object
consumption_group        object
start_time               object
quantity_kwh            float64
metering_point_count      int64
dtype: object
  price_area consumption_group                 start_time  quantity_kwh  \
0        NO1             cabin  2021-01-01 00:00:00+01:00     177071.56   
1        NO1             cabin  2021-01-01 01:00:00+01:00     171335.12   
2        NO1             cabin  2021-01-01 02:00:00+01:00     164912.02   
3        NO1             cabin  2021-01-01 03:00:00+01:00     160265.77   
4        NO1             cabin  2021-01-01 04:00:00+01:00     159828.69   

   metering_point_count  
0                100607  
1                100607  
2                100607  
3                100607  
4                100607  


In [9]:
from pymongo import MongoClient
from pymongo.server_api import ServerApi
from dotenv import load_dotenv
import os

load_dotenv()
client = MongoClient(os.getenv("MONGO_URI"), server_api=ServerApi('1'))
db = client["elhub_data"]
collection = db["consumption_data"]  # NEW TABLE (COLLECTION)

# Transform DataFrame to list of dicts and insert
data_for_mongo = df_cons.to_dict("records")
collection.insert_many(data_for_mongo)

print(f"{len(data_for_mongo)} hourly consumption records inserted (2021–2024).")

876600 hourly consumption records inserted (2021–2024).


#### In Cassandra